In [2]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = 999999999
mask_src = r'\\kukissd\Kyu_Sync\deeplab with ashley\training_set_v2\data_DL_skin_v6_control\5619_Wirtz.Denis_OTS-19_5619\view_annotations.tif'
mask = Image.open(mask_src)

In [3]:
import numpy as np
np.unique(mask)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)